In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import altair as alt
from coop_finances import *

In [3]:
TAX_RATE = 2.32
ROBBIE_PROPANE = 1135
ROBBIE_ELECTRIC = 1598
ROBBIE_HEATING_OIL = 375


def cost(property_cost, interest_rate, insurance, lawyer_fees, clt_value, assessed_value, cords_wood, price_per_cord, maintaince, rooms, propane, electric, oil, loan_duration, bookkeeping):
    return {
        "Investors": (
            lambda r, P: alt.expr.if_(
                loan_duration == 0,
                # Interest only
                r * P,
                # Fixed rate repayment
                # https://en.wikipedia.org/wiki/Mortgage_calculator#Monthly_payment_formula
                alt.expr.if_(
                    r == 0,
                    P / loan_duration,
                    (r * P) / (1 - (1 + r)**(-loan_duration)),
                ),

            )
        )(
            interest_rate / 12,
            property_cost + lawyer_fees - clt_value
        ) / rooms,
        "Land Trust": alt.expr.if_(clt_value <= 0, 0, 75 / rooms),
        "Property Tax": assessed_value * TAX_RATE / 100 / 12 / rooms,
        "Wood": cords_wood *  price_per_cord / 12 / rooms,
        "Propane": propane / 12 / rooms,
        "Electric": electric / 12 / rooms,
        "Heating Oil": oil / 12 / rooms,
        "Internet": 99 / rooms,
        "Maintenance": maintaince / 12 / rooms,
        "Insurance": insurance / 12 / rooms,
        "Bookkeeping": bookkeeping / rooms
    }

generate_plots(
    cost,
    [
        "yr = assessed_value * 2.32% (tax rate) + cords_wood * price_per_cord + propane + electric + oil + $99 (internet) + insurance + maintaince",
        "Investors = 🧮(r=interest_rate / 12, N=loan_duration * 12, P=property_cost + lawyer_fees - clt_value)",
        "monthly cost = (yr / 12 + Investors (+ $75 (Land Trust Fee) if clt_value > $0)) / rooms",
        "🧮 = https://en.wikipedia.org/wiki/Mortgage_calculator#Monthly_payment_formula",
        "source: https://github.com/saulshanabrook/coop-finances",
    ],
    rooms=Variable("🪺", "Number of residents", Range(2, 6, 1), 3, 'O'),
    property_cost=Variable("🤑", "Purchase price", Range(110 * 1000, 300 * 1000, 10 * 1000), 170 * 1000, '$'),
    interest_rate=Variable("📈", "Investor interest rate", Range(0, 0.10, 0.01), 0.03, '%'),
    loan_duration=Variable("🗓", "Loan Duration", ([5 * 12, 10 * 12, 15 * 12, 30 * 12, 0], ["5 years", "10 years", "15 years", "30 years", "Investment"]), 0, 's'),
    lawyer_fees=Variable("💼", "Lawyer fees and closing costs", Range(2000, 20000, 1000), 10000, '$'),
    clt_value=Variable("🏞", "Land value owned by CLT", Range(0, 150 * 1000, 10000), 70 * 1000, '$'),
    maintaince=Variable("🔧", "Maintenance per year", Range(0.01 * 200 * 1000, 0.05 * 200 * 1000, 0.01 * 200 * 1000), 0.03 * 200 * 1000, '$'),
    assessed_value=Variable("🇺🇸", "Assessed Value (for property taxes)", Range(110 * 1000, 300 * 1000, 5 * 1000), 130 * 1000, '$'),
    bookkeeping=Variable("📔", "Bookkeeping.coop Service Plan", ([230, 460, 1150], ["Extra Small", "Small", "Medium"]), 230, 's'),
    insurance=Variable("🏡", "Home Insurance (annual)", Range(300, 800, 100), 500, '$'),
    cords_wood=Variable("🪵", "Cords of wood per year", Range(0, 5, 1), 3, '#'),
    price_per_cord=Variable("🔥", "Price per cord wood", Range(0, 580, 10), 190, '$'),
    propane=Variable("🦕", "Propane per year", Range(0, ROBBIE_PROPANE * 2, ROBBIE_PROPANE / 10), ROBBIE_PROPANE, '$'),
    electric=Variable("🔌", "Electric per year", Range(0, ROBBIE_ELECTRIC * 2, ROBBIE_ELECTRIC / 10), ROBBIE_ELECTRIC, '$'),
    oil=Variable("🪔", "Heating oil per year", Range(0, ROBBIE_HEATING_OIL * 2, ROBBIE_HEATING_OIL / 10), ROBBIE_HEATING_OIL, '$')
)

alt.LayerChart(...)